In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My Drive/Colab Notebooks/Single-hub-location

/content/gdrive/My Drive/Colab Notebooks/Single-hub-location


In [ ]:
import os

def read_file_contents(folder_name, file_name):
    file_path = os.path.join(folder_name, file_name)

    with open(file_path, 'r') as file:
        # Merkez sayısını oku
        num_centers = int(file.readline().strip())

        # Merkez koordinatlarını oku
        center_coordinates = []
        for _ in range(num_centers):
            line = file.readline().strip().split()
            center_coordinates.append(tuple(map(float, line)))

        # Merkezler arasındaki akış miktarlarını oku
        flow_matrix = []
        for _ in range(num_centers):
            line = file.readline().strip().split()
            flow_matrix.append(list(map(float, line)))

        # Hub sayısını oku
        num_hubs = int(file.readline().strip())

        # Toplama maliyetini (collection cost) oku
        collection_cost = float(file.readline().strip())

        # Transfer maliyetini (transfer cost) oku
        transfer_cost = float(file.readline().strip())

        # Distribütör maliyetini (distribution cost) oku
        distribution_cost = float(file.readline().strip())

    return num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost

folder_name = 'AP'
file_name = '10.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)

print("Merkez Sayısı:", num_centers)
print("Merkez Koordinatları:")
for i, coords in enumerate(center_coordinates):
    print(f"Merkez {i+1}: {coords}")

'''print("\nMerkezler Arasındaki Akış Miktarları:")
for i in range(num_centers):
    for j in range(num_centers):
        print(f"Akış {i+1} -> {j+1}: {flow_matrix[i][j]}")
'''
print("\nHub Sayısı:", num_hubs)
print("Collection cost:", collection_cost)
print("Transfer cost:", transfer_cost)
print("Distribution cost:", distribution_cost)

Merkez Sayısı: 10
Merkez Koordinatları:
Merkez 1: (20355.966023, 16167.127237)
Merkez 2: (39988.59202, 19773.197847)
Merkez 3: (23572.548971, 31529.184022)
Merkez 4: (37304.36959, 32079.249435)
Merkez 5: (24512.513179, 38835.826089)
Merkez 6: (40867.253869, 38565.884488)
Merkez 7: (27520.840478, 46921.515986)
Merkez 8: (36067.877874, 44894.490748)
Merkez 9: (19345.710923, 51972.04094)
Merkez 10: (33635.749233, 49663.479404)

Hub Sayısı: 3
Collection cost: 3.0
Transfer cost: 0.75
Distribution cost: 2.0


In [ ]:
import numpy as np

 # Euclidean mesafesini hesapla
def calculate_distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

# Merkezler arasındaki mesafeleri hesapla ve costları bul
def calculate_costs(num_centers, center_coordinates):
    distances = np.zeros((num_centers, num_centers))
    for i in range(num_centers):
        for j in range(num_centers):
            distances[i, j] = calculate_distance(center_coordinates[i], center_coordinates[j])

  # Maliyet matrisini oluştur
  # ???? All of the cost are per unit (euclidean) distance, per unit flow volume.
    cost_matrix = np.zeros((num_centers, num_centers))
    for i in range(num_centers):
        for j in range(num_centers):
          #  cost_matrix[i, j] = flow_matrix[i][j] * distances[i, j]
            cost_matrix[i, j] = distances[i, j] /1000
            #cost_matrix =distances/1000 np array için

    return cost_matrix

In [ ]:
def read_solution(file_path, num_centers, num_hubs):
    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Aranacak deseni oluştur
        pattern = f"Solution for n={num_centers}, p={num_hubs}"

        # Deseni arayarak ilgili satırı bul
        target_line = None
        for line in lines:
            if line.startswith(pattern):
                target_line = line
                break

        if target_line is not None:
            # İlgili satırın altındaki iki satırı bul
            objective_line_index = lines.index(target_line)+1
            objective_value = float(lines[objective_line_index].split()[-1])  # Sadece sayıyı al

            allocation_line_index = lines.index(target_line) + 2
            allocation_line = lines[allocation_line_index].split(":")[1].strip()
            leader_hub_allocation_array = list(map(int, allocation_line.split(',')))

            return objective_value,leader_hub_allocation_array
        else:
            print(f"Belirtilen deseni içeren bir satır bulunamadı:{pattern}")
            return None

In [ ]:
def calculate_solution_matris(num_centers, hub_allocation_array, cost_matrix, transfer_cost):
  solution = [[0] * num_centers for _ in range(num_centers)]

  for i in range(num_centers):
    for j in range(num_centers):
        oi=hub_allocation_array[i]-1
        oj=hub_allocation_array[j]-1
       # print("i=",i," j=",j," oi=",oi," oj=",oj);
        solution[i][j]=cost_matrix[i][oi] + transfer_cost* cost_matrix[oi][oj] + cost_matrix[oj][j]
  return solution

In [ ]:
import random
def swap_hub(hub_location_array, hub_allocation_array):
    hub_candidates = [i+1 for i in range(len(hub_location_array)) if hub_location_array[i] == 1]
    non_hub_candidates = [i+1 for i in range(len(hub_location_array)) if hub_location_array[i] == 0]
    # Hub olan ve olmayan merkezleri rastgele seç
    new_hub = np.random.choice(non_hub_candidates)
    new_non_hub = np.random.choice(hub_candidates)

    # Hub location array güncelle
    hub_location_array[new_non_hub-1] = 0
    hub_location_array[new_hub-1] = 1

    hub_allocation_array[new_hub-1] = new_hub
    #rearrangenment
    for i in range(len(hub_allocation_array)):
        if hub_allocation_array[i] == new_non_hub:
            hub_allocation_array[i] = new_hub

    return hub_location_array, hub_allocation_array


In [ ]:
# follower için random initial solution oluştur
def random_initial_solution(num_centers, r):
  follower_hub_location_array = np.zeros(num_centers, dtype=int)
  random_indices = np.random.choice(num_centers, r, replace=False)
  follower_hub_location_array[random_indices] = 1

  follower_hubs = np.where(follower_hub_location_array == 1)[0] + 1

  follower_hub_allocation_array = np.zeros(num_centers, dtype=int)
  follower_hub_allocation_array[follower_hubs - 1] = follower_hubs

  non_hub_indices = np.where(follower_hub_location_array == 0)[0]
  random_values = np.random.choice(follower_hubs, len(non_hub_indices), replace=True)
  follower_hub_allocation_array[non_hub_indices] = random_values

  return follower_hub_location_array,follower_hub_allocation_array

In [ ]:
def calculate_fitness(num_centers,beta,gama,flow_matrix):
    fitness=0
    for i in range(num_centers):
      for j in range(num_centers):
        if gama[i][j]<beta[i][j]:
          fitness=fitness+flow_matrix[i][j]
    return fitness

In [ ]:
def calculate_total_flow(num_centers,flow_matrix):
    fitness=0
    for i in range(num_centers):
      for j in range(num_centers):
        if i!=j:
          fitness=fitness+flow_matrix[i][j]
    return fitness

In [ ]:
total_flow=calculate_total_flow(num_centers, flow_matrix)
total_flow

3557.649189999999

In [ ]:
def reallocate(hub_location_array, hub_allocation_array, k):
    # cluster oluşturma
    hub_arrays = [[] for _ in range(len(hub_location_array))]
    for i in range(len(hub_location_array)):
        if hub_location_array[i] == 1:
          for j in range(len(hub_allocation_array)):
            if hub_allocation_array[i]==hub_allocation_array[j] and i!=j:
              hub_arrays[i].append(j+1)
    # örnek hub_arrays=[[2, 3, 8, 9], [], [], [6, 10], [], [], [5], [], [], []]
    print(hub_arrays)
    non_hub_candidates = [i+1 for i in range(len(hub_location_array)) if hub_location_array[i] == 0]

    # Rastgele k tane eleman seçme
    selected_elements = random.sample(non_hub_candidates, k)
    print("Seçilen Elemanlar:", selected_elements)
    hub_candidates = [i+1 for i in range(len(hub_location_array)) if hub_location_array[i] == 1]

    for element in selected_elements:
        exclude_cluster= hub_allocation_array [element-1]
        random_cluster = random.choice([x for x in hub_candidates if x != exclude_cluster])
        hub_arrays[random_cluster-1].append(element)
        hub_arrays[exclude_cluster-1].remove(element)

        hub_allocation_array[element-1]=random_cluster

    return hub_allocation_array


In [ ]:
import math

def Algorithm(T0, Tf, Delta, Ne, beta, r):
  follower_hub_location_array,follower_hub_allocation_array=random_initial_solution(num_centers, r)
  gama=calculate_solution_matris(num_centers, follower_hub_allocation_array, cost_matrix, transfer_cost)
  T=T0
  I=0
  current_fitness=calculate_fitness(num_centers, beta, gama, flow_matrix)
  Ybest=follower_hub_allocation_array
  fbest=current_fitness
  while(T>Tf):
    new_fhub_location, new_fhub_allocation=swap_hub(follower_hub_location_array, follower_hub_allocation_array, distances)
    counter=0
    while(Ne>counter):
      counter=counter+1
      new_fhub_allocation=reallocate(new_fhub_location, new_fhub_allocation, (num_centers//2))
      new_gama=calculate_solution_matris(num_centers, new_fhub_allocation, cost_matrix, transfer_cost)
      new_fitness=calculate_fitness(num_centers, beta, new_gama, flow_matrix)
      E=new_fitness-current_fitness

      if(E>0):
        follower_hub_location_array=new_fhub_location
        follower_hub_allocation_array=new_fhub_allocation
        current_fitness=new_fitness
      else:
        p=random.random_uniform(0, 1)
        if(p>math.exp(E/T)):
          follower_hub_location_array=new_fhub_location
          follower_hub_allocation_array=new_fhub_allocation
          current_fitness=new_fitness
      if new_fitness>fbest:
        fbest=new_fitness
        Ybest=new_fhub_allocation
    T=T*Delta

  return Ybest, fbest

In [ ]:
# n=10 p=3 r=3
folder_name = 'AP'
file_name = '10.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)

print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)

cost_matrix = calculate_costs(num_centers, center_coordinates)
print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)
fitness_1=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 3)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_1.append(fbest)


gama=calculate_solution_matris(num_centers, Ybest, cost_matrix, transfer_cost)
print(fitness_1)

Merkez Sayısı: 10
p Hub Sayısı: 3
Transfer cost: 0.75
Objective Değeri:  136008.13
Leader Hub Allocation Assignment:  [3, 4, 3, 4, 7, 4, 7, 7, 7, 7]
Follower Hub Allocation Assignment: [1 2 3 2 1 1 2 3 3 2]
best fitness:  710.92868
Follower Hub Allocation Assignment: [1 1 1 4 4 1 7 1 7 4]
best fitness:  545.9860699999999
Follower Hub Allocation Assignment: [5 2 3 3 5 2 3 5 3 2]
best fitness:  636.24216
Follower Hub Allocation Assignment: [9 9 9 6 9 6 9 8 9 9]
best fitness:  521.3027700000001
Follower Hub Allocation Assignment: [1 9 1 7 7 7 7 1 9 9]
best fitness:  790.3178599999999
Follower Hub Allocation Assignment: [7 4 7 4 5 4 7 5 5 7]
best fitness:  441.3944
Follower Hub Allocation Assignment: [ 1 10  1  9  9  9  9  9  9 10]
best fitness:  443.54812000000004
Follower Hub Allocation Assignment: [ 1 10  1 10 10  8  8  8 10 10]
best fitness:  1096.15957
Follower Hub Allocation Assignment: [ 9 10  8 10  8 10  9  8  9 10]
best fitness:  942.6560700000001
Follower Hub Allocation Assignmen

In [ ]:
# n=20 p=3 r=3
folder_name = 'AP'
file_name = '20.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)

print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_2=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 3)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_2.append(fbest)
print(fitness_2)

Merkez Sayısı: 20
p Hub Sayısı: 3
Transfer cost: 0.75
Objective Değeri:  151533.08
Leader Hub Allocation Assignment:  [6, 6, 6, 12, 6, 6, 6, 12, 14, 14, 12, 12, 14, 14, 14, 12, 14, 14, 14, 14]
Follower Hub Allocation Assignment: [ 8 11  8  8  8  8 11  8 19  8 11 11 11 11 11  8 19 19 19 19]
best fitness:  951.8258000000002
Follower Hub Allocation Assignment: [ 6 17 20 17 17  6  6  6 17 17 17 20 20 17 20  6 17  6  6 20]
best fitness:  256.74721000000005
Follower Hub Allocation Assignment: [18  2  2  2 18 18 18  2  2  2 18 18 17 18  2 18 17 18 18 18]
best fitness:  472.55151
Follower Hub Allocation Assignment: [18 16 18 14 18 16 18 18 16 14 14 18 16 14 16 16 16 18 16 16]
best fitness:  476.44638999999995
Follower Hub Allocation Assignment: [ 2  2  5 13  5  2 13  2  2 13 13  2 13 13  5 13  5 13  5  2]
best fitness:  519.84349
Follower Hub Allocation Assignment: [ 6 16  7  7  6  6  7  7  7  6  6 16  6  7  6 16  7  6 16  6]
best fitness:  693.3036600000001
Follower Hub Allocation Assignment:

In [ ]:
# n=50 p=3 r=3
folder_name = 'AP'
file_name = '50.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)

print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_3=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 3)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_3.append(fbest)
print(fitness_3)

Merkez Sayısı: 50
p Hub Sayısı: 3
Transfer cost: 0.75
Objective Değeri:  158569.93
Leader Hub Allocation Assignment:  [14, 14, 14, 14, 14, 14, 14, 28, 28, 28, 14, 14, 14, 14, 14, 14, 14, 28, 28, 28, 35, 35, 35, 35, 28, 28, 28, 28, 28, 28, 35, 35, 35, 35, 35, 35, 35, 35, 28, 28, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35]
Follower Hub Allocation Assignment: [17 35 17 22 35 17 35 35 35 22 17 35 22 35 35 22 17 17 35 22 22 22 35 35
 22 35 22 22 35 35 35 17 17 17 35 35 17 22 22 35 22 22 22 35 35 35 17 22
 17 17]
best fitness:  309.15532999999994
Follower Hub Allocation Assignment: [ 4 40  3  4  3  4  4  3  4  3 40 40  3  4 40  3 40 40  4  3 40  3  3  3
  4  4  4  4  3  4 40  3  3  4  4  3  4 40 40 40  4  3 40 40 40  4 40 40
  3 40]
best fitness:  298.6484699999998
Follower Hub Allocation Assignment: [10 34 34 10 10 50 10 34 10 10 34 34 10 34 10 10 10 10 10 50 34 50 10 50
 50 10 34 10 50 50 34 50 34 34 34 50 34 50 50 10 50 50 34 34 10 10 10 50
 34 50]
best fitness:  435.20949000000013
Follower H

In [ ]:
# n=20 p=2 r=3
folder_name = 'AP'
file_name = '20.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)
num_hubs=2
print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_4=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 3)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_4.append(fbest)
print(fitness_4)

Merkez Sayısı: 20
p Hub Sayısı: 2
Transfer cost: 0.75
Objective Değeri:  172816.69
Leader Hub Allocation Assignment:  [6, 6, 6, 6, 6, 6, 6, 6, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
Follower Hub Allocation Assignment: [ 8  8  8 15  8 15  8  8 20 20  8 15  8  8 15 15 20 20  8 20]
best fitness:  714.83936
Follower Hub Allocation Assignment: [5 3 3 5 5 9 5 9 9 3 5 9 5 3 3 3 5 9 9 9]
best fitness:  568.9783800000001
Follower Hub Allocation Assignment: [19  9  3  3  9  3  9  3  9  3  9 19 19 19 19 19 19  3 19  9]
best fitness:  723.90076
Follower Hub Allocation Assignment: [15  8  8 10 15  8 15  8  8 10 10  8  8  8 15 10 15 15 10  8]
best fitness:  773.5936899999997
Follower Hub Allocation Assignment: [ 6  5  6  6  5  6 13 13 13  6  6 13 13  6 13  6  6  6 13  5]
best fitness:  614.7762499999998
Follower Hub Allocation Assignment: [ 2  2 12  7  7 12  7  7  2 12 12 12  7  2  2 12  2 12  7  7]
best fitness:  714.4629000000001
Follower Hub Allocation Assignment: [ 2  2 10 14 14 10  2  

In [ ]:
# n=20 p=4 r=3
folder_name = 'AP'
file_name = '20.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)
num_hubs=4
print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_5=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 3)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_5.append(fbest)
print(fitness_5)

Merkez Sayısı: 20
p Hub Sayısı: 4
Transfer cost: 0.75
Objective Değeri:  135624.88
Leader Hub Allocation Assignment:  [2, 2, 6, 12, 6, 6, 6, 12, 14, 14, 12, 12, 14, 14, 14, 12, 14, 14, 14, 14]
Follower Hub Allocation Assignment: [10  3  3  4 10  4  3 10 10 10  3 10  4 10 10  4 10  3  4  3]
best fitness:  776.8799099999998
Follower Hub Allocation Assignment: [ 1 19 14  1 14  1 14  1 19 14 14 19 19 14  1 19 19  1 19 14]
best fitness:  800.08121
Follower Hub Allocation Assignment: [15  3  3 15 14  3  3 15 15 14  3 14  3 14 15 14 15 15 14 14]
best fitness:  1134.9661199999996
Follower Hub Allocation Assignment: [17 17  3  4  4  3  4 17 17  4  4  3  4  4 17  4 17 17  3 17]
best fitness:  235.07868000000002
Follower Hub Allocation Assignment: [ 6  6  6  6 15  6 15  8  8  8  6  8  8  6 15  8 15 15  8 15]
best fitness:  702.8267799999999
Follower Hub Allocation Assignment: [ 2  2  2  2 10  2 10  2 19 10  2 10  2  2 19  2 19  2 19 19]
best fitness:  548.79687
Follower Hub Allocation Assignment:

In [ ]:
# n=20 p=5 r=3
folder_name = 'AP'
file_name = '20.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)
num_hubs=5
print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_6=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 3)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_6.append(fbest)
print(fitness_6)

Merkez Sayısı: 20
p Hub Sayısı: 5
Transfer cost: 0.75
Objective Değeri:  123130.09
Leader Hub Allocation Assignment:  [2, 2, 6, 12, 6, 6, 6, 12, 13, 14, 12, 12, 13, 14, 14, 12, 13, 14, 14, 14]
Follower Hub Allocation Assignment: [19 16 19 19 19 16  7 19 19  7  7 16 16 16  7 16 19  7 19  7]
best fitness:  398.4472499999999
Follower Hub Allocation Assignment: [ 4  4 16  4  7 16  7 16  7  7  4 16  4  4 16 16 16  7  7  4]
best fitness:  495.13113
Follower Hub Allocation Assignment: [16 18 16 18  7 18  7 18 16 18 16  7  7 16 18 16 18 18 18 18]
best fitness:  394.14405000000005
Follower Hub Allocation Assignment: [16 16 16 15 14 16 15 15 16 16 15 15 15 14 15 16 14 15 16 15]
best fitness:  1094.9022499999996
Follower Hub Allocation Assignment: [7 7 3 7 3 3 7 8 7 8 8 8 3 3 3 3 8 3 3 7]
best fitness:  435.23217
Follower Hub Allocation Assignment: [1 8 4 4 1 4 1 8 8 1 4 1 4 8 1 4 4 8 8 1]
best fitness:  284.5401499999998
Follower Hub Allocation Assignment: [11 14  8 14  8  8 14  8 14 14 11  8  8

In [ ]:
# n=20 p=3 r=2
folder_name = 'AP'
file_name = '20.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)

print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_7=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 2)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_7.append(fbest)
print(fitness_7)

Merkez Sayısı: 20
p Hub Sayısı: 3
Transfer cost: 0.75
Objective Değeri:  151533.08
Leader Hub Allocation Assignment:  [6, 6, 6, 12, 6, 6, 6, 12, 14, 14, 12, 12, 14, 14, 14, 12, 14, 14, 14, 14]
Follower Hub Allocation Assignment: [10 10 10 17 10 17 10 17 17 10 10 10 17 10 10 17 17 10 10 17]
best fitness:  573.7061500000001
Follower Hub Allocation Assignment: [1 4 1 4 1 4 1 4 1 1 1 1 4 1 1 1 4 1 4 4]
best fitness:  176.36245999999997
Follower Hub Allocation Assignment: [ 6 16 16 16  6  6  6  6 16  6 16  6  6  6 16 16  6  6  6  6]
best fitness:  371.70392000000004
Follower Hub Allocation Assignment: [ 5 18 18  5  5 18  5 18  5 18 18 18 18 18 18 18 18 18 18  5]
best fitness:  488.1668100000001
Follower Hub Allocation Assignment: [ 5  5  5  5  5  5 14 14 14 14 14 14  5 14  5 14 14 14 14  5]
best fitness:  657.5532899999997
Follower Hub Allocation Assignment: [16 19 16 16 16 19 19 16 19 19 19 19 19 19 16 16 19 16 19 19]
best fitness:  611.93194
Follower Hub Allocation Assignment: [12  9 12 1

In [ ]:
# n=20 p=3 r=4
folder_name = 'AP'
file_name = '20.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)

print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_8=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 4)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_8.append(fbest)
print(fitness_8)

Merkez Sayısı: 20
p Hub Sayısı: 3
Transfer cost: 0.75
Objective Değeri:  151533.08
Leader Hub Allocation Assignment:  [6, 6, 6, 12, 6, 6, 6, 12, 14, 14, 12, 12, 14, 14, 14, 12, 14, 14, 14, 14]
Follower Hub Allocation Assignment: [ 8  6  8 17  5  6  6  8  5 17  8  5 17 17  6  6 17  5 17  6]
best fitness:  488.1255100000001
Follower Hub Allocation Assignment: [ 6 10  6 10 15  6 14  6 15 10  6 14 10 14 15 15 14 10 15 10]
best fitness:  1213.90738
Follower Hub Allocation Assignment: [18 18 14 18  8  8 14  8 20 18 20 20 20 14 18 14 14 18  8 20]
best fitness:  645.0201299999999
Follower Hub Allocation Assignment: [ 2  2 15 19  2  8  2  8 15 15 15  8 15  8 15  2  2 19 19  8]
best fitness:  669.8898099999998
Follower Hub Allocation Assignment: [ 5 20 19 19  5 15 20  5 20 15 15 15  5  5 15 20  5 20 19 20]
best fitness:  800.0012500000003
Follower Hub Allocation Assignment: [16 13 12 16 12 12  7 16 13  7  7 12 13 13 12 16 13  7 12 12]
best fitness:  622.8380999999999
Follower Hub Allocation Assi

In [ ]:
# n=20 p=3 r=5
folder_name = 'AP'
file_name = '20.3'
num_centers, center_coordinates, flow_matrix, num_hubs, collection_cost, transfer_cost, distribution_cost = read_file_contents(folder_name, file_name)

print("Merkez Sayısı:", num_centers)
print("p Hub Sayısı:", num_hubs)
print("Transfer cost:", transfer_cost)

file_path = 'AP/Solutions-USApHMP.txt'
objective_value, leader_hub_allocation_array = read_solution(file_path, num_centers, num_hubs)
cost_matrix = calculate_costs(num_centers, center_coordinates)

print("Objective Değeri: ",objective_value)
print("Leader Hub Allocation Assignment: ",leader_hub_allocation_array)

beta=calculate_solution_matris(num_centers, leader_hub_allocation_array, cost_matrix, transfer_cost)

fitness_9=[]
for i in range(10):
  Ybest, fbest=Algorithm(20.000, 2000, 0.90, 25, beta, 5)
  print("Follower Hub Allocation Assignment:",Ybest)
  print("best fitness: ",fbest)
  fitness_9.append(fbest)
print(fitness_9)

Merkez Sayısı: 20
p Hub Sayısı: 3
Transfer cost: 0.75
Objective Değeri:  151533.08
Leader Hub Allocation Assignment:  [6, 6, 6, 12, 6, 6, 6, 12, 14, 14, 12, 12, 14, 14, 14, 12, 14, 14, 14, 14]
Follower Hub Allocation Assignment: [16 11 20 18 16 16 11 18 20 18 11 11 11 11 20 16 16 18 19 20]
best fitness:  704.8044400000002
Follower Hub Allocation Assignment: [13  2 14 14 14 13 16 16  2  2 11 11 13 14 11 16 14 11 11 13]
best fitness:  942.7514899999996
Follower Hub Allocation Assignment: [10 15  8  9  8  9 10  8  9 10 10  8 10 19 15  9  8 19 19 10]
best fitness:  778.9512599999998
Follower Hub Allocation Assignment: [12  3  3 10  5  3  5 10  5 10  5 12 13 12  3 10  3 10 13  3]
best fitness:  714.7924200000003
Follower Hub Allocation Assignment: [18  2  2 20 18  2 11 18 10 10 11 11  2 18 18 18 20 18 10 20]
best fitness:  561.1756899999999
Follower Hub Allocation Assignment: [ 6 20  3 16  6  6  7  6 20  3 20  3  3 16  3 16  7 20 20 20]
best fitness:  685.3531099999999
Follower Hub Allocati

In [ ]:
formatted_numbers = [f"{num:.3f}" for num in fitness_9]

# Sonuçları yazdırma
for formatted_number in formatted_numbers:
    print(formatted_number)

704.804
942.751
778.951
714.792
561.176
685.353
778.554
1086.122
627.399
528.596


In [ ]:
std_deviation = np.std(fitness_8)

# Sonucu yazdır
print("Dizin Standart Sapması:", std_deviation)

Dizin Standart Sapması: 220.66935313749187
